In [84]:
import os
import numpy as np
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import pandas as pd
import torch.nn.functional as F
from PIL import Image
import matplotlib.pyplot as plt

In [85]:
# load the model
import models
# default settings
arch = 'resnet32'
num_classes = 10
use_norm = False
model = models.__dict__[arch](num_classes=num_classes, use_norm=use_norm)

cols_names_classes = ['class_' + str(i) for i in range(0,num_classes)]
cols_names_logits = ['logit_' + str(i) for i in range(0, num_classes)]
cols_names_targets = ['target_' + str(i) for i in range(0, num_classes)]

# prepare the test data.
transform_val = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
val_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_val)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False,
        num_workers=4, pin_memory=True)

Files already downloaded and verified


In [86]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = torch.LongTensor(targets)
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]
        
        if self.transform:
            x = Image.fromarray(self.data[index])
            x = self.transform(x)
        
        return x, y
    
    def __len__(self):
        return len(self.data)

In [87]:
# load the model parameters
# vanilla training
model_ckpt_vanilla = './checkpoint/cifar10_resnet32_CE_None_exp_0.01_0/ckpt.pth.tar'
# mixup training
model_ckpt_mixup = './checkpoint/cifar10_resnet32_CE_None_exp_0.01_0_mixup/ckpt.pth.tar'
# adv training
model_ckpt_adv = './checkpoint/cifar10_resnet32_CE_None_exp_0.01_0_randaugment/ckpt.pth.tar'

In [88]:
gpu = 1
model = model.cuda(gpu)
checkpoint = torch.load(model_ckpt_vanilla, map_location = 'cuda:' + str(gpu))
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [89]:
cifarresultsdir = './cifar10results/vanilla/'
if not os.path.exists(cifarresultsdir):
    os.makedirs(cifarresultsdir)

model.eval()
logits = []
preds = []
targets = []
for i, (input, target) in enumerate(tqdm(val_loader)):
    input = input.cuda(gpu, non_blocking = True)
    logits_test = model(input)
    preds_test = F.softmax(logits_test, dim = 1)
    targets_test = F.one_hot(target, num_classes = num_classes)
    logits.append(logits_test.cpu().detach())
    preds.append(preds_test.cpu().detach())
    targets.append(targets_test)
    
logits = torch.cat(logits, dim=0)
preds = torch.cat(preds, dim=0)
targets = torch.cat(targets, dim=0)
    
df = pd.DataFrame(data=preds.numpy(), columns=cols_names_classes)
df_logits = pd.DataFrame(data=logits.numpy(), columns=cols_names_logits)
df_targets = pd.DataFrame(data=targets.numpy(), columns=cols_names_targets)
df = pd.concat([df, df_logits, df_targets], axis=1)
df.to_csv(os.path.join(cifarresultsdir, 'predictions_val.csv'), index=False)

100%|█████████████████████████████████████████████████| 10000/10000 [01:15<00:00, 133.01it/s]


In [90]:
cifarcdatadir = './CIFAR-10-C/'

alltestfiles = os.listdir(cifarcdatadir)
alltestfiles.remove('labels.npy')

labelcifar = np.load(cifarcdatadir + 'labels.npy')

# for testc in tqdm(alltestfiles):
for testc in [alltestfiles[0]]:
    
    print('Processing corruption: ', testc[:-4])
    
    datasetexample = np.load(cifarcdatadir + testc)
    
    val_datasetc = MyDataset(datasetexample, labelcifar, transform=transform_val)
    val_loaderc = torch.utils.data.DataLoader(val_datasetc, batch_size=1, shuffle=False,
        num_workers=4, pin_memory=True)
    
    savefilename = 'predictions_val_' + testc[:-4] + '.csv'
    
    model.eval()
    logits = []
    preds = []
    targets = []
    for i, (input, target) in enumerate(tqdm(val_loaderc)):
        input = input.cuda(gpu, non_blocking = True)
        logits_test = model(input)
        preds_test = F.softmax(logits_test, dim = 1)
        targets_test = F.one_hot(target, num_classes = num_classes)
        logits.append(logits_test.cpu().detach())
        preds.append(preds_test.cpu().detach())
        targets.append(targets_test)

    logits = torch.cat(logits, dim=0)
    preds = torch.cat(preds, dim=0)
    targets = torch.cat(targets, dim=0)

    df = pd.DataFrame(data=preds.numpy(), columns=cols_names_classes)
    df_logits = pd.DataFrame(data=logits.numpy(), columns=cols_names_logits)
    df_targets = pd.DataFrame(data=targets.numpy(), columns=cols_names_targets)
    df = pd.concat([df, df_logits, df_targets], axis=1)
    df.to_csv(os.path.join(cifarresultsdir, savefilename), index=False)
    

Processing corruption:  jpeg_compression


100%|█████████████████████████████████████████████████| 50000/50000 [06:40<00:00, 124.86it/s]


In [91]:
gpu = 1
model = model.cuda(gpu)
checkpoint = torch.load(model_ckpt_adv, map_location = 'cuda:' + str(gpu))
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [92]:
cifarresultsdir = './cifar10results/randaugment/'
if not os.path.exists(cifarresultsdir):
    os.makedirs(cifarresultsdir)

model.eval()
logits = []
preds = []
targets = []
for i, (input, target) in enumerate(tqdm(val_loader)):
    input = input.cuda(gpu, non_blocking = True)
    logits_test = model(input)
    preds_test = F.softmax(logits_test, dim = 1)
    targets_test = F.one_hot(target, num_classes = num_classes)
    logits.append(logits_test.cpu().detach())
    preds.append(preds_test.cpu().detach())
    targets.append(targets_test)
    
logits = torch.cat(logits, dim=0)
preds = torch.cat(preds, dim=0)
targets = torch.cat(targets, dim=0)
    
df = pd.DataFrame(data=preds.numpy(), columns=cols_names_classes)
df_logits = pd.DataFrame(data=logits.numpy(), columns=cols_names_logits)
df_targets = pd.DataFrame(data=targets.numpy(), columns=cols_names_targets)
df = pd.concat([df, df_logits, df_targets], axis=1)
df.to_csv(os.path.join(cifarresultsdir, 'predictions_val.csv'), index=False)

100%|█████████████████████████████████████████████████| 10000/10000 [01:19<00:00, 125.72it/s]


In [93]:
cifarcdatadir = './CIFAR-10-C/'

alltestfiles = os.listdir(cifarcdatadir)
alltestfiles.remove('labels.npy')

labelcifar = np.load(cifarcdatadir + 'labels.npy')

# for testc in tqdm(alltestfiles):
for testc in [alltestfiles[0]]:
    
    print('Processing corruption: ', testc[:-4])
    
    datasetexample = np.load(cifarcdatadir + testc)
    
    val_datasetc = MyDataset(datasetexample, labelcifar, transform=transform_val)
    val_loaderc = torch.utils.data.DataLoader(val_datasetc, batch_size=1, shuffle=False,
        num_workers=4, pin_memory=True)
    
    savefilename = 'predictions_val_' + testc[:-4] + '.csv'
    
    model.eval()
    logits = []
    preds = []
    targets = []
    for i, (input, target) in enumerate(tqdm(val_loaderc)):
        input = input.cuda(gpu, non_blocking = True)
        logits_test = model(input)
        preds_test = F.softmax(logits_test, dim = 1)
        targets_test = F.one_hot(target, num_classes = num_classes)
        logits.append(logits_test.cpu().detach())
        preds.append(preds_test.cpu().detach())
        targets.append(targets_test)

    logits = torch.cat(logits, dim=0)
    preds = torch.cat(preds, dim=0)
    targets = torch.cat(targets, dim=0)

    df = pd.DataFrame(data=preds.numpy(), columns=cols_names_classes)
    df_logits = pd.DataFrame(data=logits.numpy(), columns=cols_names_logits)
    df_targets = pd.DataFrame(data=targets.numpy(), columns=cols_names_targets)
    df = pd.concat([df, df_logits, df_targets], axis=1)
    df.to_csv(os.path.join(cifarresultsdir, savefilename), index=False)
    

Processing corruption:  jpeg_compression


100%|█████████████████████████████████████████████████| 50000/50000 [06:42<00:00, 124.38it/s]


In [94]:
gpu = 1
model = model.cuda(gpu)
checkpoint = torch.load(model_ckpt_mixup, map_location = 'cuda:' + str(gpu))
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [95]:
cifarresultsdir = './cifar10results/mixup/'
if not os.path.exists(cifarresultsdir):
    os.makedirs(cifarresultsdir)

model.eval()
logits = []
preds = []
targets = []
for i, (input, target) in enumerate(tqdm(val_loader)):
    input = input.cuda(gpu, non_blocking = True)
    logits_test = model(input)
    preds_test = F.softmax(logits_test, dim = 1)
    targets_test = F.one_hot(target, num_classes = num_classes)
    logits.append(logits_test.cpu().detach())
    preds.append(preds_test.cpu().detach())
    targets.append(targets_test)
    
logits = torch.cat(logits, dim=0)
preds = torch.cat(preds, dim=0)
targets = torch.cat(targets, dim=0)
    
df = pd.DataFrame(data=preds.numpy(), columns=cols_names_classes)
df_logits = pd.DataFrame(data=logits.numpy(), columns=cols_names_logits)
df_targets = pd.DataFrame(data=targets.numpy(), columns=cols_names_targets)
df = pd.concat([df, df_logits, df_targets], axis=1)
df.to_csv(os.path.join(cifarresultsdir, 'predictions_val.csv'), index=False)

100%|█████████████████████████████████████████████████| 10000/10000 [01:20<00:00, 123.50it/s]


In [96]:
cifarcdatadir = './CIFAR-10-C/'

alltestfiles = os.listdir(cifarcdatadir)
alltestfiles.remove('labels.npy')

labelcifar = np.load(cifarcdatadir + 'labels.npy')

# for testc in tqdm(alltestfiles):
for testc in [alltestfiles[0]]:
    
    print('Processing corruption: ', testc[:-4])
    
    datasetexample = np.load(cifarcdatadir + testc)
    
    val_datasetc = MyDataset(datasetexample, labelcifar, transform=transform_val)
    val_loaderc = torch.utils.data.DataLoader(val_datasetc, batch_size=1, shuffle=False,
        num_workers=4, pin_memory=True)
    
    savefilename = 'predictions_val_' + testc[:-4] + '.csv'
    
    model.eval()
    logits = []
    preds = []
    targets = []
    for i, (input, target) in enumerate(tqdm(val_loaderc)):
        input = input.cuda(gpu, non_blocking = True)
        logits_test = model(input)
        preds_test = F.softmax(logits_test, dim = 1)
        targets_test = F.one_hot(target, num_classes = num_classes)
        logits.append(logits_test.cpu().detach())
        preds.append(preds_test.cpu().detach())
        targets.append(targets_test)

    logits = torch.cat(logits, dim=0)
    preds = torch.cat(preds, dim=0)
    targets = torch.cat(targets, dim=0)

    df = pd.DataFrame(data=preds.numpy(), columns=cols_names_classes)
    df_logits = pd.DataFrame(data=logits.numpy(), columns=cols_names_logits)
    df_targets = pd.DataFrame(data=targets.numpy(), columns=cols_names_targets)
    df = pd.concat([df, df_logits, df_targets], axis=1)
    df.to_csv(os.path.join(cifarresultsdir, savefilename), index=False)
    

Processing corruption:  jpeg_compression


100%|█████████████████████████████████████████████████| 50000/50000 [06:56<00:00, 120.04it/s]


In [103]:
cifar_resultsdir = '/vol/biomedic3/zl9518/Robust-Skin-Lesion-Classification/cifar10results/vanilla/'
cnn_pred = pd.read_csv(cifar_resultsdir + 'predictions_val.csv')

In [104]:
kcls = 10
targets_all = np.array(cnn_pred[['target_0', 'target_1', 'target_2', 'target_3', 'target_4', 'target_5', 'target_6', 'target_7', 'target_8', 'target_9']])
logit_all = np.array(cnn_pred[['logit_0', 'logit_1', 'logit_2', 'logit_3', 'logit_4', 'logit_5', 'logit_6', 'logit_7', 'logit_8', 'logit_9']])
preds_all_argmax = np.argmax(logit_all, axis = 1)
# acc
target_class = np.argmax(targets_all, axis = 1)
pred_class = np.argmax(logit_all, axis = 1)
acc = np.sum(pred_class == target_class) / len(target_class)
print(acc)

0.7005


In [105]:
cifar_resultsdir = '/vol/biomedic3/zl9518/Robust-Skin-Lesion-Classification/cifar10results/randaugment/'
cnn_pred = pd.read_csv(cifar_resultsdir + 'predictions_val.csv')

In [106]:
kcls = 10
targets_all = np.array(cnn_pred[['target_0', 'target_1', 'target_2', 'target_3', 'target_4', 'target_5', 'target_6', 'target_7', 'target_8', 'target_9']])
logit_all = np.array(cnn_pred[['logit_0', 'logit_1', 'logit_2', 'logit_3', 'logit_4', 'logit_5', 'logit_6', 'logit_7', 'logit_8', 'logit_9']])
preds_all_argmax = np.argmax(logit_all, axis = 1)
# acc
target_class = np.argmax(targets_all, axis = 1)
pred_class = np.argmax(logit_all, axis = 1)
acc = np.sum(pred_class == target_class) / len(target_class)
print(acc)

0.74


In [107]:
cifar_resultsdir = '/vol/biomedic3/zl9518/Robust-Skin-Lesion-Classification/cifar10results/mixup/'
cnn_pred = pd.read_csv(cifar_resultsdir + 'predictions_val.csv')

In [108]:
kcls = 10
targets_all = np.array(cnn_pred[['target_0', 'target_1', 'target_2', 'target_3', 'target_4', 'target_5', 'target_6', 'target_7', 'target_8', 'target_9']])
logit_all = np.array(cnn_pred[['logit_0', 'logit_1', 'logit_2', 'logit_3', 'logit_4', 'logit_5', 'logit_6', 'logit_7', 'logit_8', 'logit_9']])
preds_all_argmax = np.argmax(logit_all, axis = 1)
# acc
target_class = np.argmax(targets_all, axis = 1)
pred_class = np.argmax(logit_all, axis = 1)
acc = np.sum(pred_class == target_class) / len(target_class)
print(acc)

0.7111
